# Multivector Retriever

It can often be beneficial to store multiple vectors per document. There are multiple use cases where this is beneficial. LangChain has a base MultiVectorRetriever which makes querying this type of setup easy. A lot of the complexity lies in how to create the multiple vectors per document. This notebook covers some of the common ways to create those vectors and use the MultiVectorRetriever.

The methods to create multiple vectors per document include:

    Smaller chunks: split a document into smaller chunks, and embed those (this is ParentDocumentRetriever).
    Summary: create a summary for each document, embed that along with (or instead of) the document.
    Hypothetical questions: create hypothetical questions that each document would be appropriate to answer, embed those along with (or instead of) the document.

Note that this also enables another method of adding embeddings - manually. This is great because you can explicitly add questions or queries that should lead to a document being recovered, giving you more control.

In [15]:
from langchain.retrievers.multi_vector import MultiVectorRetriever

from langchain.document_loaders import TextLoader
# from langchain.embeddings import OpenAIEmbeddings
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name = "moka-ai/m3e-base")

In [18]:
loaders = [
    TextLoader("../../docs/paul_graham_essay.txt"),
    TextLoader("../../docs/state_of_the_union.txt"),
]
docs = []
for l in loaders:
    docs.extend(l.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

# xiaodong added for shorten processing time
docs = docs[:5]

In [25]:
docs[4]

Document(page_content="Computers were expensive in those days and it took me years of nagging before I convinced my father to buy one, a TRS-80, in about 1980. The gold standard then was the Apple II, but a TRS-80 was good enough. This was when I really started programming. I wrote simple games, a program to predict how high my model rockets would fly, and a word processor that my father used to write at least one book. There was only room in memory for about 2 pages of text, so he'd write 2 pages at a time and then print them out, but it was a lot better than a typewriter.", metadata={'source': '../../docs/paul_graham_essay.txt'})

# Smaller chunks

Often times it can be useful to retrieve larger chunks of information, but embed smaller chunks. This allows for embeddings to capture the semantic meaning as closely as possible, but for as much context as possible to be passed downstream. Note that this is what the ParentDocumentRetriever does. Here we show what is going on under the hood.

In [19]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embedding_model
)
# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [20]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=300)

In [21]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [22]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [26]:
# Vectorstore alone retrieves the small chunks
retriever.vectorstore.similarity_search("word processor")[0]

Document(page_content="good enough. This was when I really started programming. I wrote simple games, a program to predict how high my model rockets would fly, and a word processor that my father used to write at least one book. There was only room in memory for about 2 pages of text, so he'd write 2 pages at a time and", metadata={'doc_id': '66eb8e17-e49e-4cff-9058-63ce1aec124f', 'source': '../../docs/paul_graham_essay.txt'})

In [28]:
# Retriever returns larger chunks
len(retriever.get_relevant_documents("word processor")[0].page_content)

557

The default search type the retriever performs on the vector database is a similarity search. LangChain Vector Stores also support searching via Max Marginal Relevance so if you want this instead you can just set the search_type property as follows:

In [29]:
from langchain.retrievers.multi_vector import SearchType

retriever.search_type = SearchType.mmr

len(retriever.get_relevant_documents("word processor")[0].page_content)

557

# Summary

Oftentimes a summary may be able to distill more accurately what a chunk is about, leading to better retrieval. Here we show how to create summaries, and then embed those.

In [30]:
import uuid

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.document import Document
from langchain.schema.output_parser import StrOutputParser

openai_api_base_address = "http://192.168.3.84:20000/v1"
chat_model = ChatOpenAI(openai_api_key = "aaabbbcccdddeeefffedddsfasdfasdf", 
    openai_api_base = openai_api_base_address,
    model_name = "chatglm3-6b-32k",
    max_retries=0)

In [31]:
chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | chat_model
    | StrOutputParser()
)

In [32]:
docs[0]

Document(page_content='What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.', metadata={'source': '../../docs/paul_graham_essay.txt'})

In [33]:
docs[0].page_content

'What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.'

In [34]:
summary = chain.invoke(docs[0])
summary

'在上大学之前，我主要在写作和编程方面工作过。我写的不是论文，而是当时认为是“入门级”的短篇小说。我的小说很糟糕，几乎没有情节，只有角色强烈的感情，我觉得这使得它们具有深度。我第一次尝试编写程序是在9年级，也就是13或14岁的时候，当时我们学校区使用IBM 1401进行数据处理。这所学校区的1401位于我们初中学校的地下室，我的朋友Rich Draves和我得到了使用它的许可。那像一个小型詹姆斯·邦德 villain的巢穴，有各种外星般的机器，包括CPU、磁盘驱动器、打印机和读卡器，都放在一个抬高的地板上，在明亮的荧光灯下。'

In [35]:
summaries = chain.batch(docs, {"max_concurrency": 5})

In [36]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries", embedding_function=embedding_model)
# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

In [37]:
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

In [38]:
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [39]:
# # We can also add the original chunks to the vectorstore if we so want
# for i, doc in enumerate(docs):
#     doc.metadata[id_key] = doc_ids[i]
# retriever.vectorstore.add_documents(docs)

In [40]:
sub_docs = vectorstore.similarity_search("word processor")

In [41]:
sub_docs[0]

Document(page_content='这篇文档描述了早期Fortran语言的使用方式。当时，用户需要将程序打印在卡片上，然后将卡片放入读卡机中，并按按钮将程序加载到内存中并运行。通常情况下，运行的结果是在极为嘈杂的打印机上打印出一些东西。', metadata={'doc_id': '83ca368b-1652-4d06-a391-4a18ce5d862a'})

In [42]:
retrieved_docs = retriever.get_relevant_documents("word processor")

In [43]:
len(retrieved_docs[0].page_content)

277

# Hypothetical Queries
An LLM can also be used to generate a list of hypothetical questions that could be asked of a particular document. These questions can then be embedded

In [44]:
functions = [
    {
        "name": "hypothetical_questions",
        "description": "Generate hypothetical questions",
        "parameters": {
            "type": "object",
            "properties": {
                "questions": {
                    "type": "array",
                    "items": {"type": "string"},
                },
            },
            "required": ["questions"],
        },
    }
]

In [45]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chat_model = ChatOpenAI(openai_api_key = "aaabbbcccdddeeefffedddsfasdfasdf", 
    openai_api_base = openai_api_base_address,
    model_name = "chatglm3-6b-32k",
    max_retries=0)

chain = (
    {"doc": lambda x: x.page_content}
    # Only asking for 3 hypothetical questions, but this could be adjusted
    | ChatPromptTemplate.from_template(
        "Generate a list of 3 hypothetical questions that the below document could be used to answer:\n\n{doc}"
    )
    | chat_model.bind(
        functions=functions, function_call={"name": "hypothetical_questions"}
    )
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

In [46]:
chain.invoke(docs[0])

OutputParserException: Could not parse function call: 'function_call'

In [ ]:
hypothetical_questions = chain.batch(docs, {"max_concurrency": 5})

In [ ]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="hypo-questions", embedding_function=embedding_model
)
# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

In [ ]:
question_docs = []
for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(
        [Document(page_content=s, metadata={id_key: doc_ids[i]}) for s in question_list]
    )

In [ ]:
retriever.vectorstore.add_documents(question_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
sub_docs = vectorstore.similarity_search("word processor")

In [ ]:
sub_docs

In [ ]:
retrieved_docs = retriever.get_relevant_documents("word processor")

In [ ]:
len(retrieved_docs[0].page_content)